In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.2'
spark_version = 'spark-3.2.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://security.ubuntu.com/ubuntu 

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge-Del2").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   25933450| RJOVP071AVAJO|0439873800|      84656342|There Was an Old ...|           Books|          5|            0|          0|   N|                Y|          Five Stars|I love it and so ...| 2015-08-31|
|         US|    1801372|R1ORGBETCDW3AI|1623953553|     729938122|      I Saw a Friend|           Books|          5|    

In [4]:
# Filter the data to retrieve all the rows where the total_votes count is equal to or greater than 20
total_votes_df = df.filter("total_votes >= 20")
total_votes_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   51227971|R3T6LR4H11XRAN|1585422959|     173554198|Parenting from th...|           Books|          5|           26|         26|   N|                Y|A MUST READ FOR A...|The reviewers tha...| 2015-08-31|
|         US|   53083708|R3QFNFMFWOQIBO|1619020653|     277126951|Jesus Land: A Memoir|           Books|          3|    

In [5]:
# Filter the new DataFrame Step 1 to retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%.
total_votes_df2 = total_votes_df.filter(total_votes_df["helpful_votes"] / total_votes_df["total_votes"] >= 0.5)
total_votes_df2.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   51227971|R3T6LR4H11XRAN|1585422959|     173554198|Parenting from th...|           Books|          5|           26|         26|   N|                Y|A MUST READ FOR A...|The reviewers tha...| 2015-08-31|
|         US|   53083708|R3QFNFMFWOQIBO|1619020653|     277126951|Jesus Land: A Memoir|           Books|          3|    

In [6]:
# Filter the DataFrame or table created in Step 2, and create a new DataFrame or table that retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'Y'.
vine_votes = total_votes_df2.filter(total_votes_df2["vine"] == "Y")
vine_votes.show()


+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   53052945|R3KKWNG5VABDYC|1594631743|     410991634|Lafayette in the ...|           Books|          5|           31|         32|   Y|                N|Bringing America'...|Sarah Vowell has ...| 2015-08-31|
|         US|   38666248|R397PIJYYVJ7PD|0385537565|     114223774|Empire of Self: A...|           Books|          4|    

In [7]:
# Repeat Step 3, but this time retrieve all the rows where the review was not part of the Vine program (unpaid), vine == 'N'.
no_vine_votes = total_votes_df2.filter(total_votes_df2["vine"] == "N")
no_vine_votes.show()


+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   51227971|R3T6LR4H11XRAN|1585422959|     173554198|Parenting from th...|           Books|          5|           26|         26|   N|                Y|A MUST READ FOR A...|The reviewers tha...| 2015-08-31|
|         US|   53083708|R3QFNFMFWOQIBO|1619020653|     277126951|Jesus Land: A Memoir|           Books|          3|    

In [10]:
# Determine the total number of paid reviews
total_paid_reviews = vine_votes.count()
total_paid_reviews

5012

In [11]:
# Determine the total number of unpaid reviews
total_unpaid_reviews = no_vine_votes.count()
total_unpaid_reviews

109297

In [21]:
# Determine the number of paid 5-star reviews
five_star_paid_reviews = vine_votes.filter(vine_votes["star_rating"]==5).count()
five_star_paid_reviews

2031

In [19]:
# Determine the number of unpaid 5-star reviews
five_star_unpaid_reviews = no_vine_votes.filter(no_vine_votes["star_rating"]==5).count()
five_star_unpaid_reviews

49967

In [20]:
# Determine the percentage of 5-star reviews - paid
paid_five_star_percent = five_star_paid_reviews.count() / total_paid_reviews * 100
paid_five_star_percent

40.52274541101357

In [22]:
# Determine the percentage of 5-star reviews - unpaid
unpaid_five_star_percent = five_star_unpaid_reviews / total_unpaid_reviews * 100
unpaid_five_star_percent

45.716716835777746